# Dependencies installation
we'll first try to fetch a pre-installed dependencies from google-drive, if it doesn't exist, we'll install it there for faster loading next *time*

In [ ]:
import numpy as np

from src import multi_run
from src.channel_simulation import ChannelSimulator
from src.split_step_fourier import SplitStepFourier
from src.visualizer import Visualizer

# Deep Learning research

## BER vs normalization factor

In [7]:
# test the impact of ber vs normalization factor for multiple n_symbols
cs = ChannelSimulator(m_qam=16, verbose=False, channel_func=SplitStepFourier())

num_realisations = 5
us_vec = multi_run.create_us_vec(n_steps=10, min_u=-2.5, max_u=2)
N_symbols_vec = np.array([64, 128, 256, 512])

with np.printoptions(precision=2):
    print(f'normalizing factors are: {us_vec}')

    bers_vec, legends = [], []
    for n_sym in N_symbols_vec:
        cs.num_symbols = n_sym
        bers, errs = multi_run.ber_vs_us(cs, us_vec, n_realisations=num_realisations)
        print(f'{n_sym} symbols: errors found = {errs} / [{cs.length_of_msg * num_realisations} '
              f'= {cs.length_of_msg} bits * {num_realisations} realisations]')
        bers_vec.append(bers)
        legends.append(f'{n_sym} symbols')

Visualizer.plot_bers(us_vec, bers_vec, legends)


normalizing factors are: [  0.0032   0.0066   0.0139   0.0291   0.0611   0.128    0.2683   0.5623
   1.1788   2.4709   5.1795  10.8571  22.7585  47.7058 100.    ]


  0%|          | 0/150 [00:00<?, ?it/s]


100%|██████████| 15/15 [00:00<00:00, 82.04it/s] 

  0%|          | 0/15 [00:00<?, ?it/s]


IndexError: list assignment index out of range

# TODOs:
* check out debug_single_with_split and explore why after split the signal looks so bad
    * probably something to do with params or code
    * idea: check with the SplitStep_full and see if the results are similar -
        * if not, there's something wrong with the code of the minimal version


Done:
* add BER at the end
* perhaps loop the BW thing further up with BER as result function
* make a graph of BER as function of normalizing factor
* make the number of symbols: 500 - 1000 check that the ber improves (power of two)
* add channel - *split step fourier algorithm* (algorithm is at agrawal 2.4.1 [2.3.46 equation without alpha] paper)
  *it includes the noise (at first we can neglect it)
* work with units of: pico second and km
* set params for split step:
    * check on symbols: [64 :power of 2 :512]
    * alpha = 0
    * maximum distance is around 1000km - need to translate to SI
    * b2 [-20 pica sec^2 / km]
    * gama = 1.27 [$km^{-1} * v^{-1}$]
    * h is the distance resolution - no larger than 0.5 km (smaller the better)
* with channel the results should be similiar if